# PSU efficiency analysis

> Version 2

Revision of the PSU analysis presented in the IMC'25 paper (Section 9)

Change log:
- v2:
  - **Fixed the interpolation function**  
  The original anaylsis used the numpy interpolation function, but that one does not extrapolate outside of the input data range, which is necessary in our case, as the load of many PSUs is smaller than the first set point for the interpolation.  
  This fix **significantly impacts** the outcome of the PSU right-sizing analysis. In particular, the cost of oversizing increases from 1% to 4% if we would replace all PSUs by the largest ones.
  - **Fixed the PSU load computation**   
  The PSU load was computed using the ratio of the median power measurements with the PSU capacity. This does not match with the standard definition of the PSU load, which is `P_out/PSU_capacity`, while our power measurements measure `P_in`, not `P_out`.
  - **Added modular analysis for efficiencies >1**  
  Some PSU efficiencies would evolve to be larger than 1, which makes no physical sense. This is because of the approximation we make in the PSU efficiency estimation. To address this, we added some modular capping of the efficiency values. The options are:
    - No capping,
    - Capping to 1  _< Default_
    - Capping to the Titanium curve.

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
from scipy.optimize import fsolve
from scipy import interpolate

import plotly.graph_objects as go

## Initialization

In [ ]:

# Select the different output format settings

# PaperPlot = True
PaperPlot = False
if PaperPlot:
    output_format = 'IMC'
else:
    output_format = 'online'

if output_format == 'online':
    font_size_px = 14
    linewidth_px = 512
    landscapewidth_px = 2*linewidth_px
    plot_path = None
    
    out_path = Path('output/online/figures')

if output_format == 'IMC':
    font_size_pt = 7
    offset = 5 # to compensate for the rounding of unit conversions
    linewidth_pt = 241 - offset  
    landscapewidth_pt = 506 - offset
    
    # 1pt = 1.333px
    font_size_px = int(font_size_pt*1.333)+1
    linewidth_px = int(linewidth_pt*1.333)+1
    landscapewidth_px = int(landscapewidth_pt*1.333)+1

    out_path = Path('output/2025_IMC/figures')

# Create the output directory if don't exist
Path(out_path).mkdir(parents=True, exist_ok=True)

# Input data
input_path = Path('input')

# Default plot layout
default_layout = {
    "title":None,
    "width":linewidth_px,
    "height":200,
    "font":{"size":font_size_px},
    "yaxis":{'title':{'font':{'size':font_size_px}}},
    "xaxis":{'title':{'font':{'size':font_size_px},
                      'text':'Time [ s ]'}}
}


## Efficient curve and 80 Plus standards

In [ ]:
# ===============
# Import PFE data and derive the numerical models 
# for the different standard by adding a constant 
# to the PFE efficiency curve
# ===============

pfe_file = Path(input_path,'pfe-efficiency-curve.csv')
df_pfe = pd.read_csv(pfe_file)
df_pfe['load_%'] = (df_pfe['load_W'] / 600)*100

# .. Model is linear interpolation between the colected points
# => The numpy function does not extrapolate outside of the input data range, 
#    which we need for our analysis.
# def eff_interp(x):
#     return np.interp(x,df_pfe['load_%']/100,df_pfe['efficiency_%']/100)

# Setup the linear interpolation between the collected points
# (which also extrapolate linearly outside of the range)
eff_interp = interpolate.interp1d(
    df_pfe['load_%']/100, 
    df_pfe['efficiency_%']/100,
    fill_value="extrapolate")

marker_opacity = 1
marker_size = 9
marker_border_size = 1
marker_line = dict(
    width=marker_border_size,
    color='black'
)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = [10,20,50,100],
        y = [90,94,96,91],
        mode='markers',
        name='Titanium',
        marker=dict(
            symbol=19, 
            line=marker_line,
            size=marker_size, 
            color='rgb(121, 121, 130)', 
            ),
    )
)
fig.add_trace(
    go.Scatter(
        x = [20,50,100],
        y = [90,94,91],
        mode='markers',
        name='Platinum',
        marker=dict(symbol=0, 
            line=marker_line,size=marker_size, color='rgb(209, 208, 206)', ),
    )
)
fig.add_trace(
    go.Scatter(
        x = [20,50,100],
        y = [88,92,88],
        mode='markers',
        name='Gold',
        marker=dict(symbol=2,
            line=marker_line, size=marker_size, color='rgb(255, 215, 0)', opacity=marker_opacity),
    )
)
fig.add_trace(
    go.Scatter(
        x = [20,50,100],
        y = [85,89,85],
        mode='markers',
        name='Silver',
        marker=dict(symbol=4,
            line=marker_line, size=marker_size, color='rgb(192, 192, 192)', opacity=marker_opacity),
    )
)
fig.add_trace(
    go.Scatter(
        x = [20,50,100],
        y = [81,85,81],
        mode='markers',
        name='Bronze',
        marker=dict(symbol=3,
            line=marker_line, size=marker_size, color='rgb(205, 127, 50)', opacity=marker_opacity),
    )
)

fig.add_trace(
    go.Scatter(
        x = df_pfe['load_%'],
        y = df_pfe['efficiency_%'],
        mode='lines',
        name='PFE600',
        marker=dict(
            color='black', 
        )
    )
)

 # y axis title
ytitle = go.layout.Annotation(
        x=-0.01,
        y=1.15,
        xref="paper",
        yref="paper",
        text="Efficiency (%)",
        showarrow=False,
        xanchor='left'
    )

# Define the custom layout options
custom_layout = dict(
    xaxis=dict(
        title=dict(
            text='Power load (%)',
            font={'size':font_size_px}
        ),
        range=[-5,105],
    ),
    yaxis=dict(
        title=None,
        range=[80,100],
    ),
    legend=dict(
        orientation='v'
    ),
    annotations=[ytitle],
    margin=dict(l=0, r=0, t=25, b=0),
)
# Combine with the defaults and apply
layout = default_layout.copy()
layout.update(custom_layout)
fig.update_layout(layout)

fig.show()
fig.write_image(out_path/'80Plus-curves.pdf')

In [ ]:
# ===============
# Interpolation functions for the different 80Plus standards
# ===============

# Function f(x) for 80PLUS Titanium standard
def t(x):
    const = 0.94 - eff_interp(0.20) + 0.02
    return eff_interp(x) + const
    # return -0.2083 * x**2 + 0.2125 * x + 0.90583

# Function f(x) for 80PLUS Platinum standard
def p(x):
    const = 0
    return eff_interp(x) + const
    # return -0.2417 * (x)**2 + 0.3025 * (x) + 0.84917

# Function f(x) for 80PLUS Gold standard
def g(x):
    const = 0.92 - eff_interp(0.50)
    return eff_interp(x) + const
    # return -0.2667 * (x)**2 + 0.32 * (x) + 0.8267

# Function f(x) for 80PLUS Silver standard
def s(x):
    const = 0.89 - eff_interp(0.50)
    return eff_interp(x) + const
    # return -0.2667 * (x)**2 + 0.32 * (x) + 0.7967

# Function f(x) for 80PLUS Bronze standard
def b(x):
    const = 0.85 - eff_interp(0.50)
    return eff_interp(x) + const
    # return -0.2667 * (x)**2 + 0.32 * (x) + 0.7567

# Function f(x) for an 80PLUS-like standard
# with custom constent term
def custom_eff(x,const):
    return eff_interp(x) + const

In [ ]:
# ===============
# Read the PSU efficiency data
# ===============

# Setup the function of the numerical solver for the load
# load_guess = 0.1 # 10% load initial guess for the solver
def load_equation(load,capacity,p_in):
    # P_out = capacity * load = p_in * eff_interp(load)
    return capacity * load - p_in * eff_interp(load)

def solve_for_load(row,PSU,label_capacity):
    load_guess = 0.1 # 10% load initial guess for the solver
    return fsolve(load_equation,load_guess, args=(row[label_capacity],row['median_power_PSU'+PSU]))[0]

def read_src_data():
        
    src_file = Path(input_path,'psu-efficiency.csv')
    df = pd.read_csv(src_file)

    # Numerically solve to get the PSU loads from the P_in values
    for PSU in ['1','2']:
        label_load = 'load_PSU' + PSU
        df[label_load] = df.apply(solve_for_load, args=(PSU,'PSU_capacity',), axis=1)           

    return df

df = read_src_data()


df


## Plot eff=f(load)

In [ ]:
# ===============
# Plot the PSU conversion efficiency data
# ===============

df = read_src_data()

fig = go.Figure()

marker_opacity = 0.5
marker_size = 9
marker_border_size = 1
marker_line = dict(
    width=marker_border_size,
    color='black'
)

# Combine the data from both PSU
loads = df['load_PSU1'].tolist()
loads.append(df['load_PSU2'].tolist())
efficiencies = df['efficiency_PSU1'].tolist()
efficiencies.append(df['efficiency_PSU2'].tolist())


fig.add_trace(
    go.Scatter(
        x = df_pfe['load_%']/100,
        y = df_pfe['efficiency_%']/100 + 0.94 - eff_interp(0.20) + 0.02,
        mode='lines',
        name='Titanium',
        marker=dict(
            color='rgb(121, 121, 130)', 
        ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = [0.2],
        y = [0.96],
        mode='markers',
        name='Titanium',
        marker=dict(
            symbol=19, 
            line=marker_line,
            size=marker_size, 
            color='rgb(121, 121, 130)', 
            ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = df_pfe['load_%']/100,
        y = df_pfe['efficiency_%']/100,
        mode='lines',
        name='Platinum',
        marker=dict(
            color='rgb(209, 208, 206)', 
        ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = [0.2],
        y = [0.922],
        mode='markers',
        name='Platinum',
        marker=dict(symbol=0, 
            line=marker_line,size=marker_size, color='rgb(209, 208, 206)', ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = df_pfe['load_%']/100,
        y = df_pfe['efficiency_%']/100 + 0.92 - eff_interp(0.50),
        mode='lines',
        name='Gold',
        marker=dict(
            color='rgb(255, 215, 0)', 
        ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = [0.20],
        y = [0.899],
        mode='markers',
        name='Gold',
        marker=dict(symbol=2,
            line=marker_line, size=marker_size, color='rgb(255, 215, 0)', 
            ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = df_pfe['load_%']/100,
        y = df_pfe['efficiency_%']/100 + 0.89 - eff_interp(0.50),
        mode='lines',
        name='Silver',
        marker=dict(
            color='rgb(192, 192, 192)', 
        ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = [0.20],
        y = [0.87],
        mode='markers',
        name='Silver',
        marker=dict(symbol=4,
            line=marker_line, size=marker_size, color='rgb(192, 192, 192)', 
            ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = df_pfe['load_%']/100,
        y = df_pfe['efficiency_%']/100 + 0.85 - eff_interp(0.50),
        mode='lines',
        name='Bronze',
        marker=dict(
            color='rgb(205, 127, 50)', 
        ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = [0.20],
        y = [0.83],
        mode='markers',
        name='Bronze',
        marker=dict(symbol=3,
            line=marker_line, size=marker_size, color='rgb(205, 127, 50)', 
            ),
        showlegend=False
    )
)


fig.add_trace(
    go.Scatter(
        x = loads,
        y = efficiencies,
        mode='markers',
        marker=dict(
            opacity=marker_opacity,
        ),
        showlegend=False
    )
)


 # y axis title
ytitle = go.layout.Annotation(
        x=-0.01,
        y=1.15,
        xref="paper",
        yref="paper",
        text="Efficiency (%)",
        showarrow=False,
        xanchor='left'
    )

# Define the custom layout options
custom_layout = dict(
    xaxis=dict(
        title=dict(
            text='Power load (%)',
            font={'size':font_size_px}
        ),
        range=[0.03,0.23],
        tickmode = 'array',
        tickvals = [0.05, 0.1, 0.15, 0.2],
        ticktext = [5, 10, 15, 20]
    ),
    yaxis=dict(
        title=None,
        range=[0.50,1.05],
    ),
    legend=dict(
        orientation='v'
    ),
    width=0.3*landscapewidth_px,
    annotations=[ytitle],
    margin=dict(l=0, r=0, t=25, b=0),
)
# Combine with the defaults and apply
layout = default_layout.copy()
layout.update(custom_layout)
fig.update_layout(layout)


fig.show()
fig.write_image(out_path/'efficiency_all.pdf')

In [ ]:
# ===============
# Highlight the data from a given router model
# ===============


# Filter for the router model of interest

# model_id = '8201-32FH'
# model_id = 'NCS-55A1-24H'
model_id = 'ASR-920-24SZ-M'
# model_id = '8201-24H8FH'
tmp = df.loc[df['router_model']== model_id]

# Combine the data from both PSU
loads = tmp['load_PSU1'].tolist()
loads.append(tmp['load_PSU2'].tolist())
efficiencies = tmp['efficiency_PSU1'].tolist()
efficiencies.append(tmp['efficiency_PSU2'].tolist())

fig.add_trace(
    go.Scatter(
        x = loads,
        y = efficiencies,
        mode='markers',
        marker=dict(
            color='red', 
            opacity=marker_opacity,
        ),
        showlegend=False
    )
)

# Define the custom layout options
custom_layout = dict(
    width=0.2*landscapewidth_px,
)
# Combine with the defaults and apply
# layout = default_layout.copy() <- Don't copy! We want to update from the previous plot
layout.update(custom_layout)
fig.update_layout(layout)

fig.write_image(out_path/str('efficiency_'+model_id+'.pdf'))

fig.show()

# Optimizing PSUs

In the following, we look at different ways we could try to optimize the power conversion efficiency of PSUs. In order to do that, we make two important assumptions:

1. All PSUs are assumed to have a power efficiency curve shaped like the one of the PFE PSU, shifted by some constant offset. 
2. For each PSU, we have one (load,efficiency) data point, which allows to calibrate the offset to the PFE curve. 

Following those assumptions lead to PSU with efficiencies larger than 1 in certain cases, which, naturally, does not make much sense. We can treat this problem a number of ways; we consider the following three.

1. `baseline` : Do not do anything; accept the non-physical approximation that some efficiencies are larger than 1.
2. `max_1` : Cap all efficiency values at 1.
3. `max_titanium` : Cap all efficiency values at the Titanium curve.

In the cell below, you can select which option to use in the analysis.

> ===  
> Effects
> - Capping the values at 1 has little effects, as the ocurences of efficiencies larger than one are rare in the dataset.
> - Capping to the Titanium curve has a more significant effect, as expected.   
> 
> ===

In [ ]:
analysis_options = [
    'baseline',
    'max_1',
    'max_titanium'
]
analysis = analysis_options[1]

def analysis_print(analysis):
    print('''==============
Analysis type: {}
=============='''.format(analysis))
    return

analysis_print(analysis)


## How much would we save with better PSUs?

That analysis is not affected by the clipping of the efficiency values to either 1 or the titanium curve, because it quantifies the savings from raising the efficiency values up to the different standards. Efficiency values above the standards do not affect the results.

In [ ]:
df = read_src_data()
tex_output = ''

# Compute the theoretic efficiency for different standards
for standard in [b,s,g,p,t]:
    for PSU in ['1','2']:
        # .. compute the efficiency
        label_eff = 'efficiency_PSU' + PSU + '_' + standard.__name__
        df[label_eff] = df['load_PSU' + PSU].apply(standard)
        df[label_eff] = df[[label_eff,'efficiency_PSU' + PSU]].max(axis=1)

        # .. derive the resulting power draw
        label_power = 'median_power_PSU' + PSU + '_' + standard.__name__
        df[label_power] = df['median_power_PSU' + PSU] * df['efficiency_PSU' + PSU] / df[label_eff]

        # .. compute the correspinding savings
        label_savings_abs = 'savings_abs_PSU' + PSU + '_' + standard.__name__
        label_savings_rel = 'savings_rel_PSU' + PSU + '_' + standard.__name__
        df[label_savings_abs] = df['median_power_PSU' + PSU] - df[label_power] 
        df[label_savings_rel] = df[label_savings_abs] / df['median_power_PSU' + PSU] 

    # .. Compute the gains
    savings_total_abs = df['savings_abs_PSU1_'+standard.__name__].sum(axis=0) +  df['savings_abs_PSU2_'+standard.__name__].sum(axis=0) 
    savings_total_rel = savings_total_abs / (df['median_power_PSU1'].sum(axis=0) + df['median_power_PSU2'].sum(axis=0))

    print('For',standard.__name__)
    print("{}\\% ({} W)\n".format(
        int(100*savings_total_rel),
        int(savings_total_abs)
        ))
    
    tex_output += '& {}\\% ({} W)'.format(
        int(100*savings_total_rel),
        int(savings_total_abs)
        )
tex_output += '\\\\'
df

print(tex_output)

## How much would we save with better-sized PSUs?

In [ ]:
df = read_src_data()
psu_capacities = sorted(df['PSU_capacity'].unique())

pd.set_option('display.max_columns', None)

overprovision_factor = [1,2,3]

def set_min_capacity(row, capacity_options, overprovision_factor):
    for c in sorted(capacity_options):
        if ((row['median_power_PSU1'] > c/overprovision_factor) or
            (row['median_power_PSU2'] > c/overprovision_factor)):
            continue
        else:
            return c
        
def set_new_capacity(row, capacity):
    if (row['min_capacity'] > capacity):
        return row['min_capacity']
    else:
        return capacity
    
def get_new_efficiency(row, PSU, label_load,analysis):
    '''
    Compute the offset between the original efficiency at the original load 
    with the efficiency given by the PFE curve. 
    -> That offset is assumed constant. 
    Using that offset, we can derive the new efficiency at the new load value
    by interpoleting from the PFE curve and adding the offset.
    '''
    offset = row['efficiency_PSU' + PSU] - eff_interp(row['load_PSU' + PSU])
    new_efficiency = custom_eff(row[label_load],offset)
    if analysis == 'baseline':
        return new_efficiency
    elif analysis == 'max_1':
        return min(new_efficiency,1)
    elif analysis == 'max_titanium':
        return min(new_efficiency,t(row['load_PSU' + PSU]))
    return custom_eff(row[label_load],offset)   

# Log
analysis_print(analysis)
    
for k in overprovision_factor:

    tex_output = ''
    print('k = {}'.format(k))
    
    # .. define the minimal capacity required per router
    df['min_capacity'] = df.apply(set_min_capacity, args=(psu_capacities, k), axis=1)

    # .. set the different cases of smallest considered capacities
    for capacity in psu_capacities:

        # .. compute the min capacity per router per case
        label_capacity = 'capacity_'+str(capacity)+'+'
        df[label_capacity] = df.apply(set_new_capacity, args=(capacity, ), axis=1)
        
        for PSU in ['1','2']:
            # .. compute the corresponding load
            label_load = 'load_PSU' + PSU + '_' + label_capacity
            df[label_load] = df.apply(solve_for_load, args=(PSU,label_capacity,), axis=1)           

            # .. compute the corresponding efficiency
            label_eff = 'efficiency_PSU' + PSU + '_' + label_capacity
            df[label_eff] = df.apply(get_new_efficiency, args=(PSU,label_load,analysis,), axis=1)

            # .. derive the resulting power draw
            label_power = 'median_power_PSU' + PSU + '_' + label_capacity
            df[label_power] = df['median_power_PSU' + PSU] * df['efficiency_PSU' + PSU] / df[label_eff]

            # .. compute the correspinding savings
            label_savings_abs = 'savings_abs_PSU' + PSU + '_' + label_capacity
            label_savings_rel = 'savings_rel_PSU' + PSU + '_' + label_capacity
            df[label_savings_abs] = df['median_power_PSU' + PSU] - df[label_power] 
            df[label_savings_rel] = df[label_savings_abs] / df['median_power_PSU' + PSU] 

        # .. Compute the total gains
        savings_total_abs = df['savings_abs_PSU1_' + label_capacity].sum(axis=0) +  df['savings_abs_PSU2_' + label_capacity].sum(axis=0) 
        savings_total_rel = savings_total_abs / (df['median_power_PSU1'].sum(axis=0) + df['median_power_PSU2'].sum(axis=0))

        print('For $',label_capacity,'$')
        print("{}\\% ({} W)\n".format(
            int(savings_total_rel*100),
            int(savings_total_abs)
            ))
        

        tex_output += '& {}\\% ({} W)'.format(
                int(100*savings_total_rel),
                int(savings_total_abs)
                )

    tex_output += '\\\\'
    print(tex_output)

    display(df)

## How much would we save by loading only one PSU instead of two?

In [ ]:
# Load the data
df = read_src_data()
tex_output = ''

analysis_print(analysis)

# .. compute the total load
df['total_power_out'] = df['median_power_PSU1']*df['efficiency_PSU1'] + df['median_power_PSU2']*df['efficiency_PSU2']
df['total_load'] = df['total_power_out'].div(df['PSU_capacity'])

# .. compute the efficiency for that load if running on only one of the PSUs
for PSU in ['1','2']:
    label_eff = 'efficiency_PSU' + PSU + '_total_load' 
    df[label_eff] = df.apply(get_new_efficiency, args=(PSU,'total_load',analysis,), axis=1)
# .. take the max
df['max_efficiency'] = df[['efficiency_PSU1_total_load','efficiency_PSU2_total_load']].max(axis=1)
# .. get the resulting total power in
df['total_power_in'] = df['total_power_out'] / df['max_efficiency']

# .. compute the savings per router
df['savings_abs'] = (df['median_power_PSU1']+ df['median_power_PSU2']) - df['total_power_in']
df['savings_rel'] = df['savings_abs'] / (df['median_power_PSU1']+ df['median_power_PSU2'])

# .. compute the global savings
savings_total_abs = df['savings_abs'].sum(axis=0)
savings_total_rel = savings_total_abs / (df['median_power_PSU1'].sum(axis=0) + df['median_power_PSU2'].sum(axis=0))


print('For using only one PSU')
print("{}\\% ({} W)\n".format(
    int(savings_total_rel*100),
    int(savings_total_abs)
    ))

tex_output += '& {}\\% ({} W)'.format(
        int(100*savings_total_rel),
        int(savings_total_abs)
        )

print(tex_output)

df

## How much would we save by loading only one PSU instead of two AND that one be of better quality?

In [ ]:
analysis_print(analysis)

# [continuing from the previous df]
tex_output = ''

# Compute the theoretic efficiency for different standards
for standard in [b,s,g,p,t]:
    # .. compute the efficiency
    label_eff = 'efficiency_' + standard.__name__
    df[label_eff] = df['total_load'].apply(standard)
    df[label_eff] = df[[label_eff,'max_efficiency']].max(axis=1)

    # .. derive the resulting power draw
    label_power = 'median_power_' + standard.__name__
    df[label_power] = df['total_power_in'] * df['max_efficiency'] / df[label_eff]

    # .. compute the savings per router
    label_savings_abs = 'savings_abs_' + standard.__name__
    label_savings_rel = 'savings_rel_' + standard.__name__
    df[label_savings_abs] = df['total_power_in'] - df[label_power] 
    df[label_savings_rel] = df[label_savings_abs] / df['total_power_in'] 

    # .. compute the global savings
    print('For using only one PSU of at least standard ',standard.__name__, '\\\\')

    # .. 1. from the first step
    savings_total_abs = df['savings_abs'].sum(axis=0)
    savings_total_rel = savings_total_abs / (df['median_power_PSU1'].sum(axis=0) + df['median_power_PSU2'].sum(axis=0))
    print("using only one {}\\% ({} W)\\\\".format(
    int(100*savings_total_rel),
    int(savings_total_abs)
    ))
    
    # .. 2. from this step alone
    savings_total_abs = df['savings_abs_'+standard.__name__].sum(axis=0)
    savings_total_rel = savings_total_abs / (df['median_power_PSU1'].sum(axis=0) + df['median_power_PSU2'].sum(axis=0))
    print("using a better one {}\\% ({} W)\\\\".format(
    int(100*savings_total_rel),
    int(savings_total_abs)
    ))

    # .. 3. all together
    savings_total_abs = df['savings_abs_'+standard.__name__].sum(axis=0) + df['savings_abs'].sum(axis=0)
    savings_total_rel = savings_total_abs / (df['median_power_PSU1'].sum(axis=0) + df['median_power_PSU2'].sum(axis=0))
    print("total: {}\\% ({} W)\n".format(
    int(100*savings_total_rel),
    int(savings_total_abs)
    ))

    tex_output += '& {}\\% ({} W)'.format(
        int(100*savings_total_rel),
        int(savings_total_abs)
        )
    
tex_output += '\\\\'
print(tex_output)

df
